## Setup: Random Seeds

In [1]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

PyTorch version: 2.8.0+cu128
Device: cuda


## Import Libraries

In [2]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

---
# QUESTION 1: Which loss function to use?

**Options:**
- `nn.MSELoss()`
- `nn.BCEWithLogitsLoss()`
- `nn.CrossEntropyLoss()`
- `nn.CosineEmbeddingLoss()`

** Answer:** `nn.BCEWithLogitsLoss()` BCEWithLogitsLoss (Binary Cross Entropy with Logits)

Used for binary classification (2 classes)
Expects 1 output neuron
Applies sigmoid internally

In [3]:
class HairTypesCNN(nn.Module):
    def __init__(self):
        super(HairTypesCNN, self).__init__()
        
        # Input: (3, 200, 200)
        # Conv2d: 32 filters, kernel 3x3, padding=0, stride=1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3, 3), padding=0, stride=1)
        self.relu1 = nn.ReLU()
        
        # MaxPool2d: 2x2
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))
        
        # After Conv2d(3x3, no padding): (200-3+1) = 198 -> (32, 198, 198)
        # After MaxPool2d(2x2): 198/2 = 99 -> (32, 99, 99)
        # Flattened: 32 * 99 * 99 = 313632
        self.flatten_size = 32 * 99 * 99
        
        # Linear layer: 64 neurons with ReLU
        self.fc1 = nn.Linear(self.flatten_size, 64)
        self.relu2 = nn.ReLU()
        
        # Output layer: 1 neuron for binary classification
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool(x)
        x = x.view(-1, self.flatten_size)
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HairTypesCNN().to(device)
print("Model created successfully")
print(model)

Model created successfully
HairTypesCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (relu2): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


---
# QUESTION 2: Number of parameters

What's the total number of parameters of the model?

**Answer:** `20073473`

In [4]:
# Option 1: Using torchsummary (uncomment if installed)
from torchsummary import summary
summary(model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


In [ ]:
# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473
